In [ ]:
import tensorflow as tf 

from keras.preprocessing.image import ImageDataGenerator


Preprocessing the Training set                 

In [ ]:
 

train_datagen = ImageDataGenerator(rescale = 1./255,                                                                  

                                   shear_range = 0.2, 

                                   zoom_range = 0.2, 

                                   horizontal_flip = True) 

training_set = train_datagen.flow_from_directory('dataset/training_set', 

                                                 target_size = (64, 64), 

                                                 batch_size = 32, 

                                                 class_mode = 'binary') 

 

rotation_range is a value in degrees (0-180), a range within which to randomly rotate pictures 

width_shift and height_shift are ranges (as a fraction of total width or height) within which to randomly translate pictures vertically or horizontally 

rescale is a value by which we will multiply the data before any other processing. Our original images consist in RGB coefficients in the 0-25 

        5, but such values would be too high for our models to process (given a typical learning rate), so we target values between 0 and 1 instead by scaling with a 1/255. factor. 

shear_range is for randomly applying shearing transformations 

zoom_range is for randomly zooming inside pictures 

horizontal_flip is for randomly flipping half of the images horizontally --relevant when there are no assumptions of horizontal assymetry (e.g. real-world pictures). 

fill_mode is the strategy used for filling in newly created pixels, which can appear after a rotation or a width/height shift. 

 

Preprocessing the Test set 

In [ ]:
 

test_datagen = ImageDataGenerator(rescale = 1./255)   test data gen is an object of imagedatagenerator classs we pre defined  

                                                       

test_set = test_datagen.flow_from_directory('dataset/test_set',      

                                            target_size = (64, 64), 

                                            batch_size = 32, 

                                            class_mode = 'binary') 

in this code we are connecting testdatagen to out test set. Low from directory is a function from imagedatagenerator class to add directory  

where test data remains.the target size argument is the size of image we fed.batc isze the number of input images we  feed 32 s a classic value  

Class mode=binary means we have binary oytcomes this is 1D , in categotrical it returns 2D one hot encoded arrays 

 

Initialising the CNN 

In [ ]:
cnn = tf.keras.models.Sequential() 

Step 1 - Convolution 

In [ ]:
 

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3])) 

#Filters means number of feature detector 

 

Step 2 - Pooling 

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2)) 

#Pool size means the size(x*x) of pool filter added over feature map. Strides means  the number of pixels we are shifting  

 

Adding a second convolutional layer 

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu')) 

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2)) 

 

#As in the first layer we don’t need to specifuy the inut size here

Step 3 - Flattening 

In [ ]:
#Before flattening all the convolutional layers are reday now we need to flatten them as a 1D array  

 cnn.add(tf.keras.layers.Flatten()) 

Step 4 - Full Connection 

In [ ]:
#Creating a fully connected layer with 128 output neurons 
cnn.add(tf.keras.layers.Dense(units=128, activation='relu')) 

Step 5 - Output Layer 

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) 
#Activation function is sigmoid because it’s a binary classifiaction problem . Softmax shoild  have been used if it was a multi class problem  

Part 3 - Training the CNN 
Compiling the CNN 

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) 

Training the CNN on the Training set and evaluating it on the Test set 

In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25) 
#Training and test sets are already defined 

Part 4 - Making a single prediction 

In [ ]:
 import numpy as np 

from keras.preprocessing import image     ----------- #function for loading single images  
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64)) ---#specifying the location and size params 
test_image = image.img_to_array(test_image)  ---- #test image is a jpg now and this library function converts it into a numpy array  
test_image = np.expand_dims(test_image, axis = 0)-- #the batch argument is necesssary but for single images it isnt possible hence add an eextra dimension 
#using expand_dims function , axis=0 menas the axis to which we are including the dimension and its 0 
result = cnn.predict(test_image)     ---this cnn is our previously built model  
training_set.class_indices       ---------prints the indices of our test classes as dogs=1 , cats=0 
if result[0][0] == 1:             ------------------the result vectior is by default 2D hence 2D indexing needed  
  prediction = 'dog' 
else: 
  prediction = 'cat' 